In [2]:
import requests
import pandas as pd
import numpy as np

In [3]:
from bs4 import BeautifulSoup

In [4]:
def series(url):
    series_url=[]
    response = requests.get(url)
    response = response.content
    soup3 = BeautifulSoup(response,'html.parser')
    all_series = soup3.find('div' , class_='cb-col-84 cb-col').findAll('div',class_='cb-srs-lst-itm')
    for series in all_series:
        series_url.append("https://www.cricbuzz.com"+series.a['href'])
    return series_url

In [5]:
def matches_url(series_url):
    match_urls=[]
    response = requests.get(series_url)
    response = response.content
    soup2 = BeautifulSoup(response, 'html.parser')
    matches = soup2.findAll('div', class_='cb-col-60 cb-col cb-srs-mtchs-tm')
    for match in matches:
        print(f"https://www.cricbuzz.com{match.a['href']}")
        match_urls.append("https://www.cricbuzz.com"+match.a['href'])
    return match_urls

In [6]:
def find_format(url):
    response = requests.get(url)
    response = response.content
    soup = BeautifulSoup(response, 'html.parser')
    if 'test' in soup.find('span', itemprop ='description')['content'].lower():
        game_format = 'Test'
    elif 'odi' in soup.find('span', itemprop ='description')['content'].lower():
        game_format = 'ODI'
    elif 't20' in soup.find('span', itemprop ='description')['content'].lower():
        game_format = 'T20'
    else:
        game_format = ''
    return game_format

In [7]:
def find_match(url):
    response = requests.get(url)
    response = response.content
    soup = BeautifulSoup(response, 'html.parser')
    match = soup.find('div', class_='cb-billing-plans-text cb-team-lft-item').h1.text
    return match

In [8]:
def find_series(url):
    response = requests.get(url)
    response = response.content
    soup = BeautifulSoup(response, 'html.parser')
    series = soup.find('div', class_='cb-nav-subhdr cb-font-12').a['title']
    return series

In [9]:
def scoreboard(url):
    response = requests.get(url)
    response = response.content
    soup = BeautifulSoup(response, 'html.parser')
    tags = soup.findAll('a', class_='cb-nav-tab')
    for tag in tags:
        if tag.text == 'Scorecard':
            print(f"https://www.cricbuzz.com{tag['href']}")
            scoreboard_url = "https://www.cricbuzz.com"+tag['href']
    return scoreboard_url
        

In [10]:
batsman_info=[]         #append the info in this
for i in range(2011,2012):
    series_urls = series(f'https://www.cricbuzz.com/cricket-scorecard-archives/{i}')  #url of each year's series
    for series_url in series_urls:
        match_urls = matches_url(series_url)    #match urls of a particular series
        for url in match_urls:
            scoreboard_url = scoreboard(url)            #Stores scoreboard url of the match
            game_format = find_format(scoreboard_url)   #Stores the format of the match
            match_name = find_match(scoreboard_url)     #Stores the match name
            series_name = find_series(scoreboard_url)   #Stores the series name
            scoreboard_response = requests.get(scoreboard_url)    
            scoreboard_response = scoreboard_response.content
            soup1 = BeautifulSoup(scoreboard_response, 'html.parser')
            location = soup1.find('a',itemprop ='location').text.strip()   #Finds and stores the location of the match
            date = soup1.find('span',itemprop ='startDate').text.strip()   #Finds and stores the date of the match
            year = soup1.find('div' , class_='cb-nav-subhdr cb-font-12').a.text.split(' ')[-1]  #Finds and stores the year of the match
            
            for i in range(1,5):                                         #Finds the info innings wise of every match
                if soup1.find('div',id = f"innings_{i}") is not None:    #We only find info of the innings present in the scoreboard
                    innings = soup1.find('div',id = f"innings_{i}")        
                    scrd_items = innings.find('div' , class_='cb-col cb-col-100 cb-ltst-wgt-hdr').findAll('div' , class_='cb-col cb-col-100 cb-scrd-itms')
                    
                    
                    '''
                    There is a possibility there might be extra div with same class name such as 'Total', 'Extras' or 'Did not bat' which are not useful
                    for us.
                    So, to remove those we check conditons
                    
                    '''
                    
                    if scrd_items[-1].find('div', class_='cb-col cb-col-27') is None:
                        scrd_items = scrd_items[:-2]
                    else:
                        scrd_items = scrd_items[:-3]
                    for pos,scrd_item in enumerate(scrd_items):
                        name = scrd_item.find('a', class_='cb-text-link').text.strip()  #Stores the name of the batsman
                        runs = scrd_item.find('div', class_='cb-col cb-col-8 text-right text-bold').text #Stores the no of runs scored
                        dismissal_mode = scrd_item.find('div', class_='cb-col cb-col-33').text.strip() #Stores the mode of dismissal
                        scrd_tags = scrd_item.findAll('div', class_='cb-col cb-col-8 text-right') 
                        for index,item in enumerate(scrd_tags):
                            if index == 0:
                                balls = (item.text.strip()) #Stores the number of balls played
                            elif index == 1:
                                fours = (item.text.strip()) #Stores the number of fours scored
                            elif index == 2:
                                sixes = (item.text.strip()) #Stores the number of sixes scored
                            else:
                                strike_rates = (item.text.strip())  #Stores the strike rate of the batsman
                        batsman_info.append([name, dismissal_mode, runs, balls, fours, sixes, strike_rates, i, location, date, pos+1, game_format, year, match_name, series_name])

C:\Users\ayush\AppData\Local\Temp\ipykernel_3392\1428399386.py:6: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  all_series = soup3.find('div' , class_='cb-col-84 cb-col').findAll('div',class_='cb-srs-lst-itm')


https://www.cricbuzz.com/cricket-scores/10711/aus-vs-ind-1st-test-india-in-australia-test-series-2011-12
https://www.cricbuzz.com/cricket-scores/10712/aus-vs-ind-2nd-test-india-in-australia-test-series-2011-12
https://www.cricbuzz.com/cricket-scores/10713/aus-vs-ind-3rd-test-india-in-australia-test-series-2011-12
https://www.cricbuzz.com/cricket-scores/10714/aus-vs-ind-4th-test-india-in-australia-test-series-2011-12


C:\Users\ayush\AppData\Local\Temp\ipykernel_3392\3114818073.py:5: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  tags = soup.findAll('a', class_='cb-nav-tab')


https://www.cricbuzz.com/live-cricket-scorecard/10711/aus-vs-ind-1st-test-india-in-australia-test-series-2011-12


C:\Users\ayush\AppData\Local\Temp\ipykernel_3392\3964278257.py:21: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  scrd_items = innings.find('div' , class_='cb-col cb-col-100 cb-ltst-wgt-hdr').findAll('div' , class_='cb-col cb-col-100 cb-scrd-itms')
C:\Users\ayush\AppData\Local\Temp\ipykernel_3392\3964278257.py:39: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  scrd_tags = scrd_item.findAll('div', class_='cb-col cb-col-8 text-right')


https://www.cricbuzz.com/live-cricket-scorecard/10712/aus-vs-ind-2nd-test-india-in-australia-test-series-2011-12
https://www.cricbuzz.com/live-cricket-scorecard/10713/aus-vs-ind-3rd-test-india-in-australia-test-series-2011-12
https://www.cricbuzz.com/live-cricket-scorecard/10714/aus-vs-ind-4th-test-india-in-australia-test-series-2011-12
https://www.cricbuzz.com/cricket-scores/10690/rsa-vs-sl-1st-test-sri-lanka-in-south-africa-2011-12
https://www.cricbuzz.com/cricket-scores/10691/rsa-vs-sl-2nd-test-sri-lanka-in-south-africa-2011-12
https://www.cricbuzz.com/cricket-scores/10692/rsa-vs-sl-3rd-test-sri-lanka-in-south-africa-2011-12
https://www.cricbuzz.com/cricket-scores/10693/rsa-vs-sl-1st-odi-sri-lanka-in-south-africa-2011-12
https://www.cricbuzz.com/cricket-scores/10694/rsa-vs-sl-2nd-odi-sri-lanka-in-south-africa-2011-12
https://www.cricbuzz.com/cricket-scores/10695/rsa-vs-sl-3rd-odi-sri-lanka-in-south-africa-2011-12
https://www.cricbuzz.com/cricket-scores/10696/rsa-vs-sl-4th-odi-sri-la

In [11]:
df = pd.DataFrame(batsman_info, columns=['Batsman', 'Mode of Dismissal', 'Runs Scored', 'Balls Faced', 'Fours', 'Sixes', 'Strike Rate', 'Innings', 'Location','Date','Batting Position', 'Format', 'Year' , 'Match', 'Series'])

In [12]:
(df[df['Format'] == ''])


,Batsman,Mode of Dismissal,Runs Scored,Balls Faced,Fours,Sixes,Strike Rate,Innings,Location,Date,Batting Position,Format,Year,Match,Series
2280,William Porterfield,c K Coetzer b J Davey,21,34,3,0,61.76,1,"Edinburgh Cricket Ground, Edinburgh","Jul 12, 10:45 AM LOCAL",1,,2011,"Scotland vs Ireland, 2nd Match - Live Cricket ...","Tri-Nation Tournament in Scotland, 2011"
2281,Paul Stirling,c R Berrington b M Haq,113,96,10,6,117.71,1,"Edinburgh Cricket Ground, Edinburgh","Jul 12, 10:45 AM LOCAL",2,,2011,"Scotland vs Ireland, 2nd Match - Live Cricket ...","Tri-Nation Tournament in Scotland, 2011"
2282,Alex Cusack,c G Goudie b Safyaan Sharif,71,83,6,0,85.54,1,"Edinburgh Cricket Ground, Edinburgh","Jul 12, 10:45 AM LOCAL",3,,2011,"Scotland vs Ireland, 2nd Match - Live Cricket ...","Tri-Nation Tournament in Scotland, 2011"
2283,Andrew Poynter,lbw b P Mommsen,51,33,6,1,154.55,1,"Edinburgh Cricket Ground, Edinburgh","Jul 12, 10:45 AM LOCAL",4,,2011,"Scotland vs Ireland, 2nd Match - Live Cricket ...","Tri-Nation Tournament in Scotland, 2011"
2284,Kevin O Brien,c M Haq b P Mommsen,0,1,0,0,0.00,1,"Edinburgh Cricket Ground, Edinburgh","Jul 12, 10:45 AM LOCAL",5,,2011,"Scotland vs Ireland, 2nd Match - Live Cricket ...","Tri-Nation Tournament in Scotland, 2011"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4076,Sachin Tendulkar,c K Sangakkara b L Malinga,18,14,2,0,128.57,2,"Wankhede Stadium, Mumbai","Apr 02, 02:30 PM LOCAL",2,,2011,"India vs Sri Lanka, Final - Live Cricket Score...",ICC World Cup 2011
4077,Gautam Gambhir,b T Perera,97,122,9,0,79.51,2,"Wankhede Stadium, Mumbai","Apr 02, 02:30 PM LOCAL",3,,2011,"India vs Sri Lanka, Final - Live Cricket Score...",ICC World Cup 2011
4078,Virat Kohli,c & b T Dilshan,35,49,4,0,71.43,2,"Wankhede Stadium, Mumbai","Apr 02, 02:30 PM LOCAL",4,,2011,"India vs Sri Lanka, Final - Live Cricket Score...",ICC World Cup 2011
4079,MS Dhoni,not out,91,79,8,2,115.19,2,"Wankhede Stadium, Mumbai","Apr 02, 02:30 PM LOCAL",5,,2011,"India vs Sri Lanka, Final - Live Cricket Score...",ICC World Cup 2011


In [13]:
batting = df.to_csv('Battng info.csv')